In [ ]:
# input data 압 축 풀 기 한 번 만 ! 
import zipfile

from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/'

output_unzip = zipfile.ZipFile("/content/drive/MyDrive/project/BreastCancer/input/open.zip", "r")  # "r": read 모드
output_unzip.extractall(file_path)
output_unzip.close()

Mounted at /content/drive


In [ ]:
import random
import pandas as pd
import numpy as np
import os
import cv2
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torchvision.models as models

from sklearn import metrics
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
import os
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if not IN_COLAB:
    os.chdir("..")
from sklearn.datasets import fetch_covtype
import random
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
%load_ext autoreload
%autoreload 2


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
import os
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Cfg, 시드 정하기!!!!!!!
EPOCHS = 40

CFG = {
    'IMG_SIZE': 512,
    'EPOCHS': EPOCHS,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':16,
    'SEED':41
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/project/BreastCancer/input/train_한글.csv")
test_df = pd.read_csv("/content/drive/MyDrive/project/BreastCancer/input/test.csv")


In [ ]:
# CAT_COL fillna
train_df['NG']= train_df['NG'].fillna(2)
train_df['HG']= train_df['HG'].fillna(2)
train_df['HG_score_1']= train_df['HG_score_1'].fillna(3)
train_df['HG_score_2']= train_df['HG_score_2'].fillna(2)
train_df['HG_score_3']= train_df['HG_score_3'].fillna(1)
train_df['DCIS_or_LCIS_type']= train_df['DCIS_or_LCIS_type'].fillna(2)
train_df['T_category']= train_df['T_category'].fillna(1)
train_df['ER']= train_df['ER'].fillna(1)
train_df['PR']= train_df['PR'].fillna(1)
train_df['HER2']= train_df['HER2'].fillna(0)
train_df['HER2_IHC']= train_df['HER2_IHC'].fillna(2)
train_df['HER2_SISH']= train_df['HER2_SISH'].fillna(0)
train_df['BRCA_mutation']= train_df['BRCA_mutation'].fillna(0)

# NUL_COL fillna
# median
train_df['암의 장경'] = train_df['암의 장경'].fillna(train_df['암의 장경'].median())
train_df['ER_Allred_score'] = train_df['ER_Allred_score'].fillna(train_df['ER_Allred_score'].median())
train_df['PR_Allred_score'] = train_df['PR_Allred_score'].fillna(train_df['PR_Allred_score'].median())
train_df['KI-67_LI_percent'] = train_df['KI-67_LI_percent'].fillna(train_df['KI-67_LI_percent'].median())
train_df['HER2_SISH_ratio'] = train_df['HER2_SISH_ratio'].fillna(train_df['HER2_SISH_ratio'].median())

In [ ]:
# CAT_COL fillna
test_df['NG']= test_df['NG'].fillna(2)
test_df['HG']= test_df['HG'].fillna(2)
test_df['HG_score_1']= test_df['HG_score_1'].fillna(3)
test_df['HG_score_2']= test_df['HG_score_2'].fillna(2)
test_df['HG_score_3']= test_df['HG_score_3'].fillna(1)
test_df['DCIS_or_LCIS_type']= test_df['DCIS_or_LCIS_type'].fillna(2)
test_df['T_category']= test_df['T_category'].fillna(1)
test_df['ER']= test_df['ER'].fillna(1)
test_df['PR']= test_df['PR'].fillna(1)
test_df['HER2']= test_df['HER2'].fillna(0)
test_df['HER2_IHC']= test_df['HER2_IHC'].fillna(2)
test_df['HER2_SISH']= test_df['HER2_SISH'].fillna(0)
test_df['BRCA_mutation']= test_df['BRCA_mutation'].fillna(0)

# NUL_COL fillna
# median
test_df['암의 장경'] = test_df['암의 장경'].fillna(test_df['암의 장경'].median())
test_df['ER_Allred_score'] = test_df['ER_Allred_score'].fillna(test_df['ER_Allred_score'].median())
test_df['PR_Allred_score'] = test_df['PR_Allred_score'].fillna(test_df['PR_Allred_score'].median())
test_df['KI-67_LI_percent'] = test_df['KI-67_LI_percent'].fillna(test_df['KI-67_LI_percent'].median())
test_df['HER2_SISH_ratio'] = test_df['HER2_SISH_ratio'].fillna(test_df['HER2_SISH_ratio'].median())

In [ ]:
train_df.isnull().sum()

ID                   0
img_path             0
mask_path            0
나이                   0
수술연월일                0
진단명                  0
암의 위치                0
암의 개수                0
암의 장경                0
NG                   0
HG                   0
HG_score_1           0
HG_score_2           0
HG_score_3           0
DCIS_or_LCIS_여부      0
DCIS_or_LCIS_type    0
T_category           0
ER                   0
ER_Allred_score      0
PR                   0
PR_Allred_score      0
KI-67_LI_percent     0
HER2                 0
HER2_IHC             0
HER2_SISH            0
HER2_SISH_ratio      0
BRCA_mutation        0
N_category           0
dtype: int64

In [ ]:
train_df, val_df, train_labels, val_labels = train_test_split(
                                                    train_df.drop(columns=['N_category']), 
                                                    train_df['N_category'], 
                                                    test_size=0.2, 
                                                    random_state=CFG['SEED']
                                                )

In [ ]:
def get_values(value):
    return value.values.reshape(-1, 1)

numeric_cols = ['나이', '암의 장경', 'ER_Allred_score', 'PR_Allred_score', 'KI-67_LI_percent', 'HER2_SISH_ratio']
ignore_cols = ['ID', 'img_path', 'mask_path', '수술연월일', 'N_category']

for col in train_df.columns:
    if col in ignore_cols:
        continue
    if col in numeric_cols:
        scaler = StandardScaler() # 전처리
        train_df[col] = scaler.fit_transform(get_values(train_df[col]))
        val_df[col] = scaler.transform(get_values(val_df[col]))
        test_df[col] = scaler.transform(get_values(test_df[col]))
    else:
        le = LabelEncoder()
        train_df[col] = le.fit_transform(get_values(train_df[col]))
        val_df[col] = le.transform(get_values(val_df[col]))
        test_df[col] = le.transform(get_values(test_df[col]))

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, medical_df, labels, transforms=None):
        self.medical_df = medical_df
        self.transforms = transforms
        self.labels = labels
        
    def __getitem__(self, index):
        img_path = self.medical_df['img_path'].iloc[index]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
                
        if self.labels is not None:
            tabular = torch.Tensor(self.medical_df.drop(columns=['ID', 'img_path', 'mask_path', '수술연월일']).iloc[index])
            label = self.labels[index]
            return image, tabular, label
        else:
            tabular = torch.Tensor(self.medical_df.drop(columns=['ID', 'img_path', '수술연월일']).iloc[index])
            return image, tabular
        
    def __len__(self):
        return len(self.medical_df)

In [ ]:
train_transforms = A.Compose([
                            A.HorizontalFlip(),
                            A.VerticalFlip(),
                            A.Rotate(limit=90, border_mode=cv2.BORDER_CONSTANT,p=0.3),
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transforms = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
train_dataset = CustomDataset(train_df, train_labels.values, train_transforms)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_df, val_labels.values, test_transforms)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
import os
 
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [ ]:
##gdrive path 설정
GPATH = '/content/drive/MyDrive/project/BreastCancer/baseline/'
print(GPATH)
createFolder(GPATH)


print('########################################################')
class ImgFeatureExtractor(nn.Module):
    def __init__(self):
        super(ImgFeatureExtractor, self).__init__()
        #####모델 입력하기 까먹지 말긔#####
        self.backbone = models.resnext50_32x4d(pretrained=True)
        self.embedding = nn.Linear(1000,512)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.embedding(x)
        return x

print('########################################################')    
class TabularFeatureExtractor(nn.Module):
    def __init__(self):
        super(TabularFeatureExtractor, self).__init__()
        self.embedding = nn.Sequential(
            nn.Linear(in_features=23, out_features=128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Linear(in_features=128, out_features=256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Linear(in_features=256, out_features=512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
            nn.Linear(in_features=512, out_features=512)
        )
        
    def forward(self, x):
        x = self.embedding(x)
        return x
print('########################################################')    
class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.img_feature_extractor = ImgFeatureExtractor()
        self.tabular_feature_extractor = TabularFeatureExtractor()
        self.classifier = nn.Sequential(
            nn.Linear(in_features=1024, out_features=1),
            nn.Sigmoid(),
        )
        
    def forward(self, img, tabular):
        img_feature = self.img_feature_extractor(img)
        tabular_feature = self.tabular_feature_extractor(tabular)
        feature = torch.cat([img_feature, tabular_feature], dim=-1)
        output = self.classifier(feature)
        return output
print('########################################################')
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    dic_log={
        'epoch':[],
        'train_loss':[],
        'val_loss':[],
        'val_score':[]
    }
    model.to(device)
    criterion = nn.BCEWithLogitsLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for img, tabular, label in tqdm(iter(train_loader)):
            img = img.float().to(device)
            tabular = tabular.float().to(device)
            label = label.float().to(device)
            
            optimizer.zero_grad()
            
            model_pred = model(img, tabular)
            
            loss = criterion(model_pred, label.reshape(-1,1))
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
            tl = np.mean(train_loss)
            
        val_loss, val_score = validation(model, criterion, val_loader, device)
        print(f'Epoch [{epoch}], Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}] Val Score : [{val_score:.5f}]')
        dic_log['epoch'].append(epoch)
        dic_log['train_loss'].append(np.mean(train_loss))
        dic_log['val_loss'].append(val_loss)
        dic_log['val_score'].append(val_score)

        if scheduler is not None:
            scheduler.step(val_score)
        
        if best_score < val_score:
            best_score = val_score
            best_model = model
            # best 뽑기
            test_dataset = CustomDataset(test_df, None, test_transforms)
            test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
            torch.save(model, GPATH+f'{MODEL}/{VERSION}/ckp/best_{MODEL}_{epoch}.pt') # 각 epoch가 끝날 때마다 모델 상태 저장
            predictions,preds = inference(best_model, test_loader, device)
            print('########################################################')
            submit = pd.read_csv('./sample_submission.csv')
            submit['N_category'] = preds
            submit['predictions'] = predictions
            submit.to_csv(GPATH+f'{MODEL}/{VERSION}/submit/best_submit_{MODEL}_{EPOCHS}_{epoch}.csv', index=False)


        torch.save(model, GPATH+f'{MODEL}/{VERSION}/ckp/checkpoint_{MODEL}_{epoch}.pt') # 각 epoch가 끝날 때마다 모델 상태 저장
        print('########################################################')
        submit = pd.read_csv('./sample_submission.csv')
        predictions,preds = inference(model, test_loader, device)
        submit['N_category'] = preds
        submit['predictions'] = predictions
        submit.to_csv(GPATH+ f'{MODEL}/{VERSION}/submit/submit_{MODEL}_{epoch}.csv', index=False)

    df_log = pd.DataFrame(dic_log)
    df_log.to_csv(GPATH+ f'{MODEL}/{VERSION}/log_{MODEL}_{VERSION}.csv',index=False)
    return best_model

print('########################################################')
def validation(model, criterion, val_loader, device):
    model.eval()
    pred_labels = []
    true_labels = []
    val_loss = []
    threshold = 0.4
    with torch.no_grad():
        for img, tabular, label in tqdm(iter(val_loader)):
            true_labels += label.tolist()
            
            img = img.float().to(device)
            tabular = tabular.float().to(device)
            label = label.float().to(device)
            
            model_pred = model(img, tabular)
            
            loss = criterion(model_pred, label.reshape(-1,1))
            
            val_loss.append(loss.item())
            
            model_pred = model_pred.squeeze(1).to('cpu')  
            pred_labels += model_pred.tolist()


    pred_labels = np.where(np.array(pred_labels) > threshold, 1, 0)
    val_score = metrics.f1_score(y_true=true_labels, y_pred=pred_labels, average='macro')


    return np.mean(val_loss), val_score

print('########################################################')

def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    threshold = 0.4
    
    with torch.no_grad():
        for img, tabular in tqdm(iter(test_loader)):
            img = img.float().to(device)
            tabular = tabular.float().to(device)
            
            model_pred = model(img, tabular)
            
            model_pred = model_pred.squeeze(1).to('cpu')
            
            preds += model_pred.tolist()
    
    predictions=np.array(preds)
    preds = np.where(np.array(preds) > threshold, 1, 0)
    
    return predictions,preds


/content/drive/MyDrive/project/BreastCancer/baseline/
########################################################
########################################################
########################################################
########################################################
########################################################
########################################################



# MODEL_21

In [ ]:
print(CFG)

{'IMG_SIZE': 512, 'EPOCHS': 40, 'LEARNING_RATE': 0.0001, 'BATCH_SIZE': 16, 'SEED': 41}


In [ ]:
MODEL = 'resnext50_32x4d'
VERSION = 21
createFolder(GPATH+'/'+MODEL+'/'+str(VERSION))
createFolder(GPATH+'/'+MODEL+'/'+str(VERSION)+'/ckp/')
createFolder(GPATH+'/'+MODEL+'/'+str(VERSION)+'/submit/')
print(GPATH+'/'+MODEL+'/'+str(VERSION))

device = torch.device("cuda")
model = nn.DataParallel(ClassificationModel())

# 학습 이어 하기
#PATH = '/content/drive/MyDrive/project/BreastCancer/baseline/resnext50_32x4d/1/ckp/best_resnext50_32x4d_9.pt'
#model = torch.load(PATH)
model.eval()
optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1, threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

test_dataset = CustomDataset(test_df, None, test_transforms)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

predictions,preds = inference(infer_model, test_loader, device)


submit = pd.read_csv('./sample_submission.csv')
submit['N_category'] = preds
submit['predictions'] = predictions
submit.to_csv(f'/content/drive/MyDrive/project/BreastCancer/baseline/{MODEL}/{VERSION}/submit/{MODEL}_{EPOCHS}.csv', index=False)

/content/drive/MyDrive/project/BreastCancer/baseline//resnext50_32x4d/21


Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


  0%|          | 0.00/95.8M [00:00<?, ?B/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

Epoch [40], Train Loss : [0.54766] Val Loss : [0.60928] Val Score : [0.76367]
########################################################


  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]